## Install Libraries

In [1]:
!pip install -U bitsandbytes
!pip install -U transformers
!pip install -U peft
!pip install -U accelerate
!pip install -U trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 81.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 69.4 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 22.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install tensorflow==2.11

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 77.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 114.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 48.8 MB/s eta 0:00:00
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.2
    Uninstalling tensorboard-data-server-0.7.2:
      Successfully uninstalled tensorboard-data-server-0.7.2
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: keras
    Found existing installation: ker

In [3]:
import os,torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging
from transformers import TrainingArguments, set_seed
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from trl import SFTTrainer
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
set_seed(42)
import random
import pandas as pd
import numpy as np
from tqdm import tqdm
from huggingface_hub import login

## Model Load

In [4]:
my_model = "jsmount/Fine-tuning-gemma-Design"

In [5]:
login()

In [6]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
    ) # 4bit quantization

# 토크나이저 불러오기
tokenizer = AutoTokenizer.from_pretrained(my_model, padding_side="left")

# 모델 불러오기
model = AutoModelForCausalLM.from_pretrained(
        my_model,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
    )
model.config.use_cache = True
model.eval()

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/34.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/41.6M [00:00<?, ?B/s]

Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 2304, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x Gemma2DecoderLayer(
        (self_attn): Gemma2Attention(
          (q_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=2304, out_features=2048, bias=False)
            (lora_dropout): ModuleDict(
              (default): Identity()
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=2304, out_features=8, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=8, out_features=2048, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=2304, out_features=1024, bias=False)
            (lora_dropout): ModuleDict(
         

## Test

In [7]:
test = pd.read_csv('/content/drive/MyDrive/Gemma/Data/Design/Design_vdata.csv')

In [8]:
test.head()

,성별,나이대,신입,질문,답변
0,여성,50대,경력,매일 공부하고 있는 분야가 있나요,어 매일 공부하고 있는 분야가 있나요 라고 하셨는데요. 저는 어 공부하는 것이 있습...
1,여성,50대,경력,대학 생활 가장 힘들었던 경험은 무엇인가요,어 대학 생활 중에서 가장 힘들었던 경험은 무엇인가요 라고 하셨는데요. 저는 방송통...
2,여성,50대,경력,지금 제일 가고 싶은 회사는,지금 제일 가고 싶은 회사는 이라고 질문을 주셨는데요. 물론 제가 지금 일하고 있는...
3,여성,50대,경력,아 기존 회사를 그만둔 것 퇴사한 것에 대해서 현재 후회가 없으신지 궁금하네요 어 ...,퇴사한 것을 후회하지 않으시나요 라고 질문을 주셨는데 어 간혹 가다가 그만두기 전에...
4,여성,50대,경력,어 내가 직무를 수행함에 있어서 어떤 뭐 모르는 과제도 나올 수 있겠죠 그런 문제에...,직무 수행 시 모르는 과제에 대해서 해결하는 방법은 무엇입니까 라고 질문을 주셨는데...


In [9]:
test.shape

(813, 5)

In [10]:
import os,torch

# 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 출력
print('Device:', device)  # 출력결과: cuda
print('Count of using GPUs:', torch.cuda.device_count())

Device: cuda
Count of using GPUs: 1


In [11]:
questions = []
preds = []

def generate100(age, career, gender, question):
    text = f"### {age}의 {career}의 {gender}에게 질문: {question} \n ### 답변:"
    inputs = tokenizer(text, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_new_tokens=100)
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    parsed_ans = decoded.split('### 답변: ')[1]
    return parsed_ans

In [12]:
print(generate100(age='20대', career='신입', gender='남성', question='회사 생활에서 가장 중요한 것은 무엇이라고 생각하나요?'))

The 'max_batch_size' argument of HybridCache is deprecated and will be removed in v4.46. Use the more precisely named 'batch_size' argument instead.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


회사 생활에서 가장 중요한 것은 저는 팀워크라고 생각합니다. 팀워크가 중요하다고 생각하는 이유는 팀워크가 없으면 어떤 일을 할 수 없다는 생각을 합니다. 팀워크가 없으면 어떤 일을 할 수 없다는 생각을 하기 때문에 팀워크가 가장 중요하다고 생각합니다. 팀워크가 없으면 어떤 일을 할 수 없다는 생각을
